In [2]:
import os 
import sys 
sys.path.append("..")
import argparse

import random
import math
import numpy as np
import torch
import torch.nn as nn
from torch.optim import SGD, lr_scheduler, AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm

from data.augmentations import get_transform
from data.get_datasets import get_datasets, get_class_splits

# import SSB
# from SSB.get_datasets.get_gcd_datasets_funcs import get_gcd_datasets
# from SSB.utils import load_class_splits

from util.general_utils import AverageMeter, init_experiment
from util.cluster_and_log_utils import log_accs_from_preds
from config import exp_root
from model import DINOHead, info_nce_logits, SupConLoss, DistillLoss, ContrastiveLearningViewGenerator, get_params_groups, CustomDINOHead

from pytorch_metric_learning import losses

In [20]:
# backbone = torch.hub.load('facebookresearch/dino:main', 'dino_xcit_medium_24_p8')
backbone = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


In [21]:
# !pip install ptflops


In [22]:
head_type = None #"custom"
# feat_dim = 512
feat_dim = 768
# mlp_out_dim = 200 
# mlp_out_dim = 196
mlp_out_dim = 100
num_mlp_layers = 3

if head_type == "custom":
    projector = CustomDINOHead(in_dim=feat_dim, out_dim=mlp_out_dim, nlayers=num_mlp_layers)
else: 
    projector = DINOHead(in_dim=feat_dim, out_dim=mlp_out_dim, nlayers=num_mlp_layers)
    
model = nn.Sequential(backbone, projector)

In [23]:
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(model, (3, 224, 224), as_strings=True,print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Sequential(
  91.88 M, 98.847% Params, 21.96 GMac, 100.000% MACs, 
  (0): DinoVisionTransformer(
    85.51 M, 91.991% Params, 21.96 GMac, 99.971% MACs, 
    (patch_embed): PatchEmbed(
      452.35 k, 0.487% Params, 115.8 MMac, 0.527% MACs, 
      (proj): Conv2d(452.35 k, 0.487% Params, 115.8 MMac, 0.527% MACs, 3, 768, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity(0, 0.000% Params, 0.0 Mac, 0.000% MACs, )
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        7.09 M, 7.625% Params, 1.82 GMac, 8.287% MACs, 
        (norm1): LayerNorm(1.54 k, 0.002% Params, 197.38 KMac, 0.001% MACs, (768,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          2.36 M, 2.541% Params, 606.34 MMac, 2.761% MACs, 
          (qkv): Linear(1.77 M, 1.906% Params, 454.76 MMac, 2.070% MACs, in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(0, 0.000% Params, 0.0 Mac, 0.000% MACs, p=0.0, inplace=False)
          (proj): Linear

In [15]:
21.96 * 2

43.92

In [24]:
%%timeit
model(torch.randn(1, 3, 224, 224))

50 ms ± 91.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
208 ms
50 ms

SyntaxError: invalid syntax (989647429.py, line 1)

In [ ]:
50 ms